## Read me

1. ``pip install tensorflow``

2. Put data files in
``cancer-net/pnet/_database``
which can be downloaded from Google Drive [here](https://drive.google.com/uc?id=17nssbdUylkyQY1ebtxsIw5UzTAd0zxWb&export=download).

3. To install pysr, use ``conda install -c conda-forge pysr``, then run ``python -c 'import pysr; pysr.install()'``



In [1]:
import os
import imp
import tensorflow as tf
import pandas as pd
import numpy as np

import pysr

from cancernet.pnet.config_path import PROSTATE_LOG_PATH, POSTATE_PARAMS_PATH
from cancernet.pnet.model.model_factory import get_model
from cancernet.pnet.model.nn import Model
from cancernet.pnet.model.builders.builders_utils import get_map_from_layer, get_layer_maps

from cancernet.pnet.data.data_access import Data
from cancernet.pnet.data.prostate_paper.data_reader import ProstateDataPaper, load_data_type
from cancernet.pnet.data.prostate_paper import data_reader
from cancernet.pnet.data.pathways.reactome import ReactomeNetwork


from sklearn.metrics import roc_auc_score, roc_curve, auc, average_precision_score, f1_score, accuracy_score, \
    precision_score, recall_score

/tmp/ipykernel_654211/4137107731.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2022-07-15 12:24:29.768872: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-15 12:24:29.772278: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /mnt/sw/nix/store/f0ycdncw8dw4wlicnzm74lgv9c51rlg4-openblas-0.3.20/lib:/cm/shared/apps/slurm/current/lib64
2022-07-15 12:24:29.772288: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
params_file = './pnet/onsplit_average_reg_10_tanh_large_testing'
log_dir = os.path.join(PROSTATE_LOG_PATH, params_file)
log_dir = log_dir
params_file = os.path.join(POSTATE_PARAMS_PATH, params_file)

params_file_full = params_file + '.py'
print(params_file_full)
params = imp.load_source(params_file, params_file_full)

/mnt/home/ttesileanu/Documents/research/deep-cancer-conda/cancernet/pnet/train/params/P1000/./pnet/onsplit_average_reg_10_tanh_large_testing.py


In [3]:
# manually adjusted P-net model params
n_hidden_layers = 2
params.models[0]['params']['model_params']['n_hidden_layers'] = n_hidden_layers
params.models[0]['params']['model_params']['w_reg'] = [0.001] * (n_hidden_layers+1)
params.models[0]['params']['model_params']['w_reg_outcomes'] = [0.001] * (n_hidden_layers+1)
params.models[0]['params']['model_params']['dropout'] = [0.5]+[0.1] * (n_hidden_layers+1)
params.models[0]['params']['model_params']['loss_weights'] = [2] * (n_hidden_layers-6) + [2, 7, 20, 54, 148, 400]

In [4]:
# model inherits a sklearn BaseEstimator
model = Model(**params.models[0]['params'])
model.get_params()

{'model_params': {'use_bias': True,
  'w_reg': [0.001, 0.001, 0.001],
  'w_reg_outcomes': [0.001, 0.001, 0.001],
  'dropout': [0.5, 0.1, 0.1, 0.1],
  'loss_weights': [2, 7, 20, 54, 148, 400],
  'optimizer': 'Adam',
  'activation': 'tanh',
  'data_params': {'id': 'ALL',
   'type': 'prostate_paper',
   'params': {'data_type': ['mut_important', 'cnv_del', 'cnv_amp'],
    'drop_AR': False,
    'cnv_levels': 3,
    'mut_binary': True,
    'balanced_data': False,
    'combine_type': 'union',
    'use_coding_genes_only': True,
    'selected_genes': 'tcga_prostate_expressed_genes_and_cancer_genes.csv',
    'training_split': 0}},
  'add_unk_genes': False,
  'shuffle_genes': False,
  'kernel_initializer': 'lecun_uniform',
  'n_hidden_layers': 2,
  'attention': False,
  'dropout_testing': False},
 'fitting_params': {'samples_per_epoch': 10,
  'select_best_model': False,
  'monitor': 'val_o6_f1',
  'verbose': 2,
  'epoch': 300,
  'shuffle': True,
  'batch_size': 50,
  'save_name': 'pnet',
  'debug

In [5]:
ret = model.build_fn(**model.model_params)
pnet_mod, feature_names, reactome_map = ret

{'id': 'ALL', 'type': 'prostate_paper', 'params': {'data_type': ['mut_important', 'cnv_del', 'cnv_amp'], 'drop_AR': False, 'cnv_levels': 3, 'mut_binary': True, 'balanced_data': False, 'combine_type': 'union', 'use_coding_genes_only': True, 'selected_genes': 'tcga_prostate_expressed_genes_and_cancer_genes.csv', 'training_split': 0}}
n_hidden_layers 2


/mnt/home/ttesileanu/Documents/research/deep-cancer-conda/cancernet/pnet/data/prostate_paper/data_reader.py:69: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  x = x.loc[:, intersect]
/mnt/home/ttesileanu/Documents/research/deep-cancer-conda/cancernet/pnet/data/prostate_paper/data_reader.py:69: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  x = x.loc[:, intersect]
/mnt/home/ttesileanu/Documents/research/deep-cancer-conda/cancernet/pnet/data/prostate_paper/data_reader.py:69: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  x = x.loc[:, intersect]


(1011, 27687)
(1011, 1)
(1011,)
(27687,)
input dimension 27687 self.units 9229
n_inputs_per_node 3
self.kernel_initializer <keras.regularizers.L2 object at 0x7f79facdd3d0> <keras.initializers.initializers_v2.LecunUniform object at 0x7f79facddac0> <keras.regularizers.L2 object at 0x7f79facdd3d0>
input dimensions (None, 27687)


2022-07-15 12:24:45.020710: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /mnt/sw/nix/store/f0ycdncw8dw4wlicnzm74lgv9c51rlg4-openblas-0.3.20/lib:/cm/shared/apps/slurm/current/lib64
2022-07-15 12:24:45.020739: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-07-15 12:24:45.021096: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX51

pathways 146
genes 10621
pathways 26
genes 147
pathways 1
genes 26
original dropout [0.5, 0.1, 0.1, 0.1]
dropout [1, 2] [0.5, 0.1, 0.1, 0.1] [0.001, 0.001, 0.001]
layer 0, dropout  0.1 w_reg 0.001
layer 1, dropout  0.1 w_reg 0.001
Compiling...
loss_weights [2, 7, 20, 54, 148, 400]
[<keras.engine.input_layer.InputLayer object at 0x7f7b4033d700>, <cancernet.pnet.model.layers_custom.Diagonal object at 0x7f79facdd040>, <keras.layers.regularization.dropout.Dropout object at 0x7f79facdddf0>, <cancernet.pnet.model.layers_custom.SparseTF object at 0x7f79f73caf40>, <keras.layers.regularization.dropout.Dropout object at 0x7f79fa63a490>, <cancernet.pnet.model.layers_custom.SparseTF object at 0x7f79f6affac0>, <keras.layers.core.dense.Dense object at 0x7f79fa644cd0>, <keras.layers.core.dense.Dense object at 0x7f79f6534610>, <keras.layers.core.dense.Dense object at 0x7f79f6532910>, <keras.layers.core.activation.Activation object at 0x7f79fa621eb0>, <keras.layers.core.activation.Activation object at 

In [6]:
tf.keras.utils.plot_model(pnet_mod)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [7]:
data = Data(**params.data[0])

x_train, x_validate_, x_test_, y_train, y_validate_, y_test_, info_train, info_validate_, info_test_, cols = data.get_train_validate_test()

print(x_train.shape)

/mnt/home/ttesileanu/Documents/research/deep-cancer-conda/cancernet/pnet/data/prostate_paper/data_reader.py:69: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  x = x.loc[:, intersect]
/mnt/home/ttesileanu/Documents/research/deep-cancer-conda/cancernet/pnet/data/prostate_paper/data_reader.py:69: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  x = x.loc[:, intersect]
/mnt/home/ttesileanu/Documents/research/deep-cancer-conda/cancernet/pnet/data/prostate_paper/data_reader.py:69: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  x = x.loc[:, intersect]


(807, 27687)


In [8]:
pnet_mod.fit(
    x_train, y_train,
    validation_data=(x_validate_, y_validate_),
    epochs=20,
    batch_size=10,
    verbose=0
)

input dimensions (None, 27687)
input dimensions (None, 27687)
input dimensions (None, 27687)


In [9]:
y_hat = pnet_mod.predict(x_test_)
print([y.shape for y in y_hat])

input dimensions (None, 27687)
4/4 [==============================] - 0s 5ms/step
[(102, 1), (102, 1), (102, 1)]


In [10]:
ys = y_test_.flatten()
outs = y_hat[-1]

fpr_valid, tpr_valid, _ = roc_curve(ys, outs)
test_auc = auc(fpr_valid, tpr_valid)

print('accuracy', accuracy_score(ys, outs>0.5))
print('auc', test_auc)
print('aupr', average_precision_score(ys, outs))
print('f1', f1_score(ys, outs>0.5))
print('precision', precision_score(ys, outs>0.5))
print('recall', recall_score(ys, outs>0.5))

accuracy 0.8823529411764706
auc 0.9524221453287197
aupr 0.912863896559413
f1 0.806451612903226
precision 0.8928571428571429
recall 0.7352941176470589


In [11]:
layer_dict = {x.name:x for x in pnet_mod.layers}
explain_mod = tf.keras.models.Model(
    inputs=pnet_mod.inputs,
    outputs=[layer_dict['h0'].output, layer_dict['h1'].output, layer_dict['h2'].output, layer_dict['o_linear3'].output]
)

In [12]:
reactome = ReactomeNetwork()
pathway_names = reactome.reactome.pathway_names.copy()
pathway_names.index = pathway_names['reactome_id']
pathway_names['pathway_name'] = pathway_names['pathway_name'].str.replace(' ','_').str.replace('-','_')

In [13]:
data_for_pysr = explain_mod.predict(x_train)
print(len(data_for_pysr))

input dimensions (None, 27687)
26/26 [==============================] - 0s 5ms/step
4


In [14]:
h0 = pd.DataFrame(data_for_pysr[0], columns=reactome_map[0].index)
h1 = pd.DataFrame(data_for_pysr[1], columns=pathway_names.loc[reactome_map[1].index]['pathway_name'])
h2 = pd.DataFrame(data_for_pysr[2], columns=pathway_names.loc[reactome_map[2].index]['pathway_name'])
out = pd.DataFrame(data_for_pysr[3], columns=['out'])

In [15]:
h1 = (h1-h1.mean())/(h1.std())

In [16]:
tmp = h1.join(out)
tmp_corr = tmp.corr()['out'].abs().sort_values(ascending=False).drop('out')
tmp_corr.head(20)

Post_translational_protein_modification         0.950862
RNA_Polymerase_II_Transcription                 0.941000
Cellular_responses_to_stress                    0.820384
Signaling_by_Rho_GTPases                        0.811195
Innate_Immune_System                            0.787191
Adaptive_Immune_System                          0.767294
Metabolism_of_lipids                            0.742517
Cytokine_Signaling_in_Immune_system             0.715097
Membrane_Trafficking                            0.707136
Intracellular_signaling_by_second_messengers    0.695092
Signaling_by_GPCR                               0.689032
Axon_guidance                                   0.672813
Signaling_by_Receptor_Tyrosine_Kinases          0.639966
Signaling_by_WNT                                0.624448
Signaling_by_NOTCH                              0.623344
Infectious_disease                              0.618394
Diseases_of_signal_transduction                 0.606040
Cell_Cycle_Checkpoints         

In [17]:
tmp[tmp_corr.head(20).index].corr().round(3)

,Post_translational_protein_modification,RNA_Polymerase_II_Transcription,Cellular_responses_to_stress,Signaling_by_Rho_GTPases,Innate_Immune_System,Adaptive_Immune_System,Metabolism_of_lipids,Cytokine_Signaling_in_Immune_system,Membrane_Trafficking,Intracellular_signaling_by_second_messengers,Signaling_by_GPCR,Axon_guidance,Signaling_by_Receptor_Tyrosine_Kinases,Signaling_by_WNT,Signaling_by_NOTCH,Infectious_disease,Diseases_of_signal_transduction,Cell_Cycle_Checkpoints,Metabolism_of_amino_acids_and_derivatives,Death_Receptor_Signalling
Post_translational_protein_modification,1.000,0.914,-0.859,0.841,-0.722,0.688,0.647,-0.601,-0.664,0.621,0.614,-0.587,0.526,0.517,-0.548,0.531,-0.532,0.545,0.501,-0.467
RNA_Polymerase_II_Transcription,0.914,1.000,-0.857,0.833,-0.648,0.674,0.629,-0.569,-0.562,0.653,0.558,-0.563,0.514,0.537,-0.571,0.474,-0.560,0.523,0.493,-0.436
Cellular_responses_to_stress,-0.859,-0.857,1.000,-0.930,0.497,-0.494,-0.437,0.455,0.416,-0.491,-0.463,0.381,-0.344,-0.406,0.481,-0.369,0.360,-0.489,-0.419,0.360
Signaling_by_Rho_GTPases,0.841,0.833,-0.930,1.000,-0.530,0.489,0.465,-0.431,-0.461,0.414,0.516,-0.407,0.402,0.427,-0.404,0.403,-0.347,0.337,0.395,-0.493
Innate_Immune_System,-0.722,-0.648,0.497,-0.530,1.000,-0.712,-0.621,0.602,0.768,-0.510,-0.641,0.718,-0.661,-0.622,0.520,-0.666,0.520,-0.454,-0.603,0.583
Adaptive_Immune_System,0.688,0.674,-0.494,0.489,-0.712,1.000,0.642,-0.568,-0.687,0.686,0.628,-0.616,0.594,0.581,-0.493,0.616,-0.513,0.453,0.556,-0.559
Metabolism_of_lipids,0.647,0.629,-0.437,0.465,-0.621,0.642,1.000,-0.599,-0.637,0.592,0.608,-0.622,0.566,0.517,-0.415,0.521,-0.486,0.442,0.481,-0.463
Cytokine_Signaling_in_Immune_system,-0.601,-0.569,0.455,-0.431,0.602,-0.568,-0.599,1.000,0.569,-0.557,-0.570,0.509,-0.461,-0.444,0.575,-0.527,0.384,-0.597,-0.383,0.502
Membrane_Trafficking,-0.664,-0.562,0.416,-0.461,0.768,-0.687,-0.637,0.569,1.000,-0.416,-0.639,0.704,-0.623,-0.551,0.424,-0.706,0.416,-0.426,-0.609,0.549
Intracellular_signaling_by_second_messengers,0.621,0.653,-0.491,0.414,-0.510,0.686,0.592,-0.557,-0.416,1.000,0.450,-0.452,0.524,0.468,-0.582,0.405,-0.537,0.634,0.386,-0.370


In [18]:
# Dataset
X_for_pysr = tmp[tmp_corr.head(20).index]
y_for_pysr = data_for_pysr[3]

default_pysr_params = dict(
    populations=30,
    procs=4,
    model_selection="best",
)


In [19]:
# Learn equations
model = pysr.PySRRegressor(
        niterations=30,
        binary_operators=["plus", "sub", "mult", "div", "greater"],
        unary_operators=["neg", "square"],
        **default_pysr_params)

model.fit(X_for_pysr, y_for_pysr)

/mnt/home/ttesileanu/miniconda3/envs/cancerenv/lib/python3.9/site-packages/pysr/sr.py:1068: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/mnt/home/ttesileanu/miniconda3/envs/cancerenv/lib/python3.9/site-packages/pysr/sr.py:1581: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. Consider using feature selection techniques to select the most important features (you can do this automatically with the `select_k_features` parameter), or, alternatively, doing a dimensionality reduction beforehand. For example, `X = PCA(n_components=6).fit_transform(X)`, using scikit-learn's `PCA` class, will reduce the number of features to 6 in an interpretable way, as each resultant feature will be a linear combination of the original features. 
  warnings.warn(
  Activating project at `~/miniconda3/envs/cancerenv/share/julia/environments/pysr-0.

Started!

Cycles per second: 4.680e+04
Head worker occupation: 23.7%
Progress: 129 / 900 total iterations (14.333%)
Hall of Fame:
-----------------------------------------
Complexity  loss       Score     Equation
1           7.866e+00  3.679e-01  Post_translational_protein_modification
3           3.522e+00  4.017e-01  (RNA_Polymerase_II_Transcription * 2.616294)
4           3.207e+00  9.383e-02  (neg(Post_translational_protein_modification) * -2.7517922)
5           1.612e+00  6.878e-01  ((RNA_Polymerase_II_Transcription * 2.616294) + -1.3821228)
6           1.281e+00  2.298e-01  ((neg(Post_translational_protein_modification) * -2.7690861) + -1.3820808)
7           6.851e-01  6.260e-01  (((RNA_Polymerase_II_Transcription * 2.526082) - Innate_Immune_System) + -1.3820792)
9           4.595e-01  1.997e-01  (((Post_translational_protein_modification + RNA_Polymerase_II_Transcription) * 1.3522842) - (Cytokine_Signaling_in_Immune_system - -1.3927))
11          4.353e-01  2.708e-02  ((RNA_P

PySRRegressor.equations_ = [
	    pick     score                                           equation  \
	0         0.000000            Post_translational_protein_modification   
	1         0.482035  (Post_translational_protein_modification * 3.2...   
	2   >>>>  0.506444  ((Post_translational_protein_modification * 3....   
	3         0.231896  (((RNA_Polymerase_II_Transcription * 2.526082)...   
	4         0.200141  ((((Post_translational_protein_modification + ...   
	5         0.057009  ((((Post_translational_protein_modification + ...   
	6         0.020223  (((((Post_translational_protein_modification +...   
	7         0.315560  neg((((Post_translational_protein_modification...   
	8         0.020839  neg((((Post_translational_protein_modification...   
	9         0.209210  neg(((((RNA_Polymerase_II_Transcription - Cyto...   
	10        0.074724  neg((((((RNA_Polymerase_II_Transcription - Cyt...   
	11        0.013244  neg((((((RNA_Polymerase_II_Transcription - Cyt...   
	
	        loss  complexity  
	0   7.865608           1  
	1   2.999450           3  
	2   1.089307           5  
	3   0.685059           7  
	4   0.459079           9  
	5   0.409609          11  
	6   0.393373          13  
	7   0.286918          14  
	8   0.281001          15  
	9   0.227955          16  
	10  0.196311          18  
	11  0.191180          20  
]

In [20]:
model.equations_.sort_values("score", ascending=False)[['complexity', 'loss', 'score', 'equation']]

,complexity,loss,score,equation
2,5,1.089307,0.506444,((Post_translational_protein_modification * 3....
1,3,2.999450,0.482035,(Post_translational_protein_modification * 3.2...
7,14,0.286918,0.315560,neg((((Post_translational_protein_modification...
3,7,0.685059,0.231896,(((RNA_Polymerase_II_Transcription * 2.526082)...
9,16,0.227955,0.209210,neg(((((RNA_Polymerase_II_Transcription - Cyto...
4,9,0.459079,0.200141,((((Post_translational_protein_modification + ...
10,18,0.196311,0.074724,neg((((((RNA_Polymerase_II_Transcription - Cyt...
5,11,0.409609,0.057009,((((Post_translational_protein_modification + ...
8,15,0.281001,0.020839,neg((((Post_translational_protein_modification...
6,13,0.393373,0.020223,(((((Post_translational_protein_modification +...


In [21]:
model.sympy(9)

-0.4265413*Cytokine_Signaling_in_Immune_system - 0.4265413*Innate_Immune_System + 0.4265413*Metabolism_of_lipids + Post_translational_protein_modification + 1.4265413*RNA_Polymerase_II_Transcription - 1.3515584

In [22]:
model.sympy()

3.207414*Post_translational_protein_modification - 1.3819706